## Load Base Model

In [2]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from pruna.algorithms.smasher_config import SmasherConfig
from pruna.smash import smash

tokenizer = AutoTokenizer.from_pretrained('HuggingFaceH4/zephyr-7b-beta')
model = AutoModelForCausalLM.from_pretrained('HuggingFaceH4/zephyr-7b-beta', trust_remote_code=True, torch_dtype="auto")
model.to('cuda')
ins = tokenizer("Who is the 45th president of the USA ?", return_tensors="pt", truncation=True).to('cuda')

Post-training Optimization Tool is deprecated and will be removed in the future. Please use Neural Network Compression Framework instead: https://github.com/openvinotoolkit/nncf
Nevergrad package could not be imported. If you are planning to use any hyperparameter optimization algo, consider installing it using pip. This implies advanced usage of the tool. Note that nevergrad is compatible only with Python 3.7+


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


## Smash it!

### Define Config

In [3]:
smasher_config = SmasherConfig()
smasher_config['quantizer'] = 'huggingface-gptq'
smasher_config['n_quantization_bits'] = 4
smasher_config['tokenizer_name'] = tokenizer

### Smash

In [4]:
smashed_model = smash(
        model=model,
        data_module="Polyglot_1000",
        api_key='your-api-key',
        model_config=None,
        smasher_config=smasher_config,
        device='cuda',
    )

Received a 500 error, retrying in 3 seconds...
[ WARNING ] Found cached dataset parquet (/nfs/homedirs/rachwan/.cache/huggingface/datasets/Polyglot-or-Not___parquet/Polyglot-or-Not--Fact-Completion-bf9cfaebc2874386/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Quantize...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Quantizing model.layers blocks :   0%|          | 0/32 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Success.


In [5]:
smashed_model.to('cuda')

## Base Model Generation

In [7]:
%%time
results = model.generate(**ins, max_length=30)

CPU times: user 1.28 s, sys: 45 ms, total: 1.33 s
Wall time: 1.99 s


In [8]:
output = tokenizer.decode(results[0])

In [9]:
output

'<s> Who is the 45th president of the USA ?\n\nDonald John Trump is the 45th and current president of the United'

## Smashed Model Generation

In [10]:
%%time
results = smashed_model(ins.input_ids, max_length=30)

CPU times: user 976 ms, sys: 27.4 ms, total: 1 s
Wall time: 1.5 s


In [11]:
output = tokenizer.decode(results[0])

In [14]:
output

'<s> Who is the 45th president of the USA ?\nGroupLayout:\nThe 45th president of the USA is Donald John Trump'